In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)
quiet_library(ArchR)

In [ ]:
library(colorspace)

In [ ]:
trim_cd8_na <- readRDS(file = '../Figure4/trim_cd8_na.rds')

In [ ]:
sp_merge <- readRDS(file = '/home/jupyter/Organized_Gating_Analysis/Seurat_Objects/sp_merge_gating_celltype_updated.rds')

In [ ]:
mnp <- readRDS('mnp_analysis_3wnn_updated.rds')

In [ ]:
table(mnp$cell.types)

In [ ]:
mnp <- SetIdent(mnp, value = 'cell.types')
mnp2_only <- subset(mnp, idents = 'MNP-2')
mnp2_only$pure_celltype <- 'CD8 MNP-2'

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'gating_celltype')
cd8_nonnaive <- subset(sp_merge, idents = c('CD8 TEMRA'))
cd8_nonnaive$pure_celltype <- cd8_nonnaive$gating_celltype

In [ ]:
trim_cd8_na <- SetIdent(trim_cd8_na, value = 'pure_celltype')
cd8na <- subset(trim_cd8_na, idents = 'CD8 Naive')

In [ ]:
# remove any overlapping cells
cd8na_clean <- subset(cd8na, cells = colnames(mnp2_only[['RNA']]), invert = TRUE)
cd8na_clean
cd8na

In [ ]:
# remove any overlapping cells
cd8_nonnaive_clean <- subset(cd8_nonnaive, cells = colnames(mnp2_only[['RNA']]), invert = TRUE)
cd8_nonnaive_clean
cd8_nonnaive

In [ ]:
cd8_merge <- merge(cd8na_clean, c(cd8_nonnaive_clean,mnp2_only))

In [ ]:
cd8_merge <- NormalizeData(cd8_merge, assay = 'ADT', margin = 2, normalization.method = 'CLR')
cd8_merge <- SetIdent(cd8_merge, value = 'pure_celltype')

In [ ]:
cd8_split <- SplitObject(cd8_merge, split.by = 'pure_celltype')

In [ ]:
mnp_adt_avg_exp <- lapply(cd8_split, function(x){
    avg_exp <- AverageExpression(x, group.by = 'pbmc_sample_id', assay = 'ADT', features = c('CD27','CD244','CD11b'))
    return(avg_exp)
})

In [ ]:
write.csv(mnp_adt_avg_exp$`CD8 MNP-2`, file = 'mnp2_avg_adt_exp.csv')
write.csv(mnp_adt_avg_exp$`CD8 Naive`, file = 'cd8na_avg_adt_exp.csv')
write.csv(mnp_adt_avg_exp$`CD8 TEMRA`, file = 'cd8temra_avg_adt_exp.csv')

In [ ]:
adt_norm_mtx_list <- lapply(cd8_split, function(x){
    mtx <- x@assays$ADT@data
    return(mtx)
})

In [ ]:
# write.csv(adt_norm_mtx_list$`CD8 Naive`, file = 'adt_cd8na_norm_mtx.csv')
# write.csv(adt_norm_mtx_list$`CD8 MNP-2`, file = 'adt_cd8mnp2_norm_mtx.csv')
# write.csv(adt_norm_mtx_list$`CD8 TEMRA`, file = 'adt_cd8temra_norm_mtx.csv')

In [ ]:
cd8_merge <- SetIdent(cd8_merge, value = 'pure_celltype')
cd8_merge <- subset(cd8_merge, downsample = 2500)

In [ ]:
cd8_merge <- NormalizeData(cd8_merge, assay = 'RNA')
cd8_merge <- NormalizeData(cd8_merge, assay = 'ADT', margin = 2, normalization.method = 'CLR')
cd8_merge <- SetIdent(cd8_merge, value = 'pure_celltype')

In [ ]:
df_build <- function(seurat_object){
    meta <- seurat_object@meta.data
    meta$TCRValpha7.2 <- as.vector(seurat_object@assays$ADT['TCR-Valpha7.2',])
    meta$CD161 <- as.vector(seurat_object@assays$ADT['CD161',])
    meta$CD197 <- as.vector(seurat_object@assays$ADT['CD197',])
    meta$CD45RA <- as.vector(seurat_object@assays$ADT['CD45RA',])
    meta$CD27 <- as.vector(seurat_object@assays$ADT['CD27',])
    meta$CD244 <- as.vector(seurat_object@assays$ADT['CD244',])
    meta$CD11b <- as.vector(seurat_object@assays$ADT['CD11b',])
    return(meta)}

In [ ]:
test_df <- df_build(cd8_merge)

In [ ]:
# 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 12)
test_df %>% 
    mutate(pure_celltype = factor(pure_celltype, levels = c('CD8 Naive','CD8 MNP-2','CD8 TEMRA'))) %>%
    ggplot(aes(x = pure_celltype, y = CD27, color=pure_celltype)) + 
      ggdist::stat_halfeye(
        aes(color = pure_celltype,
        fill = pure_celltype),
        adjust = .5, 
        width = .6, 
        .width = 0, 
        justification = -.2#, 
        # point_colour = NA
      ) + 
      geom_boxplot(
        width = .15, 
        outlier.shape = NA
      ) +
      ## add justified jitter from the {gghalves} package
      gghalves::geom_half_point(
        ## draw jitter on the left
        side = "l", 
        ## control range of jitter
        range_scale = .4, 
        ## add some transparency
        alpha = .3
      ) +
      coord_cartesian(xlim = c(1.2, NA), clip = "off") + 
      scale_color_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      scale_fill_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      theme_bw() +
      theme(axis.text.x = element_text(size = 20),
            axis.title.x = element_blank(),
            axis.text.y = element_text(size = 20),
            axis.title.y = element_blank(),
            legend.position = 'none',
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank())

In [ ]:
# Open a pdf file
pdf("plots/cd27_raindrop_2.pdf", width = 8, height = 12) 
# 2. Create a plot
test_df %>% 
    mutate(pure_celltype = factor(pure_celltype, levels = c('CD8 Naive','CD8 MNP-2','CD8 TEMRA'))) %>%
    ggplot(aes(x = pure_celltype, y = CD27, color=pure_celltype)) + 
      ggdist::stat_halfeye(
        aes(color = pure_celltype,
        fill = pure_celltype),
        adjust = .5, 
        width = .6, 
        .width = 0, 
        justification = -.2#, 
        # point_colour = NA
      ) + 
      geom_boxplot(
        width = .15, 
        outlier.shape = NA
      ) +
      ## add justified jitter from the {gghalves} package
      gghalves::geom_half_point(
        ## draw jitter on the left
        side = "l", 
        ## control range of jitter
        range_scale = .4, 
        ## add some transparency
        alpha = .3
      ) +
      coord_cartesian(xlim = c(1.2, NA), clip = "off") + 
      scale_color_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      scale_fill_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      theme_bw() +
      theme(axis.text.x = element_text(size = 20),
            axis.title.x = element_blank(),
            axis.text.y = element_text(size = 20),
            axis.title.y = element_blank(),
            legend.position = 'none',
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank())
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 12)
test_df %>% 
    mutate(pure_celltype = factor(pure_celltype, levels = c('CD8 Naive','CD8 MNP-2','CD8 TEMRA'))) %>%
    ggplot(aes(x = pure_celltype, y = CD244, color=pure_celltype)) + 
      ggdist::stat_halfeye(
        aes(color = pure_celltype,
        fill = pure_celltype),
        adjust = .5, 
        width = .6, 
        .width = 0, 
        justification = -.2#, 
        # point_colour = NA
      ) + 
      geom_boxplot(
        width = .15, 
        outlier.shape = NA
      ) +
      ## add justified jitter from the {gghalves} package
      gghalves::geom_half_point(
        ## draw jitter on the left
        side = "l", 
        ## control range of jitter
        range_scale = .4, 
        ## add some transparency
        alpha = .3
      ) +
      coord_cartesian(xlim = c(1.2, NA), clip = "off") + 
      scale_color_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      scale_fill_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      theme_bw() +
      theme(axis.text.x = element_text(size = 20),
            axis.title.x = element_blank(),
            axis.text.y = element_text(size = 20),
            axis.title.y = element_blank(),
            legend.position = 'none',
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank())

In [ ]:
# Open a pdf file
pdf("plots/cd244_raindrop_2.pdf", width = 8, height = 12) 
# 2. Create a plot
test_df %>% 
    mutate(pure_celltype = factor(pure_celltype, levels = c('CD8 Naive','CD8 MNP-2','CD8 TEMRA'))) %>%
    ggplot(aes(x = pure_celltype, y = CD244, color=pure_celltype)) + 
      ggdist::stat_halfeye(
        aes(color = pure_celltype,
        fill = pure_celltype),
        adjust = .5, 
        width = .6, 
        .width = 0, 
        justification = -.2#, 
        # point_colour = NA
      ) + 
      geom_boxplot(
        width = .15, 
        outlier.shape = NA
      ) +
      ## add justified jitter from the {gghalves} package
      gghalves::geom_half_point(
        ## draw jitter on the left
        side = "l", 
        ## control range of jitter
        range_scale = .4, 
        ## add some transparency
        alpha = .3
      ) +
      coord_cartesian(xlim = c(1.2, NA), clip = "off") + 
      scale_color_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      scale_fill_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      theme_bw() +
      theme(axis.text.x = element_text(size = 20),
            axis.title.x = element_blank(),
            axis.text.y = element_text(size = 20),
            axis.title.y = element_blank(),
            legend.position = 'none',
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank())
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 12)
test_df %>% 
    mutate(pure_celltype = factor(pure_celltype, levels = c('CD8 Naive','CD8 MNP-2','CD8 TEMRA'))) %>%
    ggplot(aes(x = pure_celltype, y = CD11b, color=pure_celltype)) + 
      ggdist::stat_halfeye(
        aes(color = pure_celltype,
        fill = pure_celltype),
        adjust = .5, 
        width = .6, 
        .width = 0, 
        justification = -.2#, 
        # point_colour = NA
      ) + 
      geom_boxplot(
        width = .15, 
        outlier.shape = NA
      ) +
      ## add justified jitter from the {gghalves} package
      gghalves::geom_half_point(
        ## draw jitter on the left
        side = "l", 
        ## control range of jitter
        range_scale = .4, 
        ## add some transparency
        alpha = .3
      ) +
      coord_cartesian(xlim = c(1.2, NA), clip = "off") + 
      scale_color_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      scale_fill_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      theme_bw() +
      theme(axis.text.x = element_text(size = 20),
            axis.title.x = element_blank(),
            axis.text.y = element_text(size = 20),
            axis.title.y = element_blank(),
            legend.position = 'none',
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank())

In [ ]:
# Open a pdf file
pdf("plots/cd11b_raindrop_2.pdf", width = 8, height = 12) 
# 2. Create a plot
test_df %>% 
    mutate(pure_celltype = factor(pure_celltype, levels = c('CD8 Naive','CD8 MNP-2','CD8 TEMRA'))) %>%
    ggplot(aes(x = pure_celltype, y = CD11b, color=pure_celltype)) + 
      ggdist::stat_halfeye(
        aes(color = pure_celltype,
        fill = pure_celltype),
        adjust = .5, 
        width = .6, 
        .width = 0, 
        justification = -.2#, 
        # point_colour = NA
      ) + 
      geom_boxplot(
        width = .15, 
        outlier.shape = NA
      ) +
      ## add justified jitter from the {gghalves} package
      gghalves::geom_half_point(
        ## draw jitter on the left
        side = "l", 
        ## control range of jitter
        range_scale = .4, 
        ## add some transparency
        alpha = .3
      ) +
      coord_cartesian(xlim = c(1.2, NA), clip = "off") + 
      scale_color_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      scale_fill_manual(values = c('#807dba','#1f78b4','#EF67EB')) + 
      theme_bw() +
      theme(axis.text.x = element_text(size = 20),
            axis.title.x = element_blank(),
            axis.text.y = element_text(size = 20),
            axis.title.y = element_blank(),
            legend.position = 'none',
            panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank())
# Close the pdf file
dev.off() 